# Usage reference guide for haystack-reverse

this is an example of every haystack-reverse commands.

The zeus.vmem.856.dump is there https://dl.dropboxusercontent.com/u/10222931/HAYSTACK/zeus.vmem.856.dump.tgz

It was extracted from pid 856 from the zeus.img image from http://malwarecookbook.googlecode.com/svn-history/r26/trunk/17/1/zeus.vmem.zip

In [1]:
!haystack-reverse --help

usage: haystack-reverse [-h] [--debug | --quiet] [--interactive] [--nommap] dump_folder_name

Reverse the data structure from the process memoryThe process dump is a folder produced by a haystack-dump script.

positional arguments:
  dump_folder_name  Use this memory dump folder

optional arguments:
  -h, --help        show this help message and exit
  --debug           Set verbosity to DEBUG
  --quiet           Set verbosity to ERROR only
  --interactive     drop to python command line after action
  --nommap          disable mmap()-ing


First we need to generate the analysis for the process memory dump.

In [3]:
!haystack-reverse ../test/dumps/vol/zeus.vmem.856.dump

INFO:reverse.api:Reversing Fields
INFO:model:[+] <FieldReverser>: START
INFO:model:[+] <FieldReverser>: START on heap 0x90000
INFO:model:[+] <FieldReverser>: START on heap 0x190000
INFO:model:[+] <FieldReverser>: START on heap 0x1a0000
INFO:model:[+] <FieldReverser>: START on heap 0x350000
INFO:model:[+] <FieldReverser>: START on heap 0x3b0000
INFO:model:[+] <FieldReverser>: START on heap 0x460000
INFO:model:[+] <FieldReverser>: START on heap 0xc30000
INFO:model:[+] <FieldReverser>: START on heap 0xd60000
INFO:model:[+] <FieldReverser>: START on heap 0xe20000
INFO:model:[+] <FieldReverser>: START on heap 0xe80000
INFO:model:[+] <FieldReverser>: START on heap 0x7f6f0000
INFO:reverse.api:Fixing Text Fields
INFO:model:[+] <TextFieldCorrection>: START
INFO:model:[+] <TextFieldCorrection>: START on heap 0x90000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x190000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x1a0000
INFO:model:[+] <TextFieldCorrection>: START on heap 0x350000

Then we can start to use some of the cli

In [5]:
!ls -al ../test/dumps/vol/zeus.vmem.856.dump/cache/

total 4536
drwxrwxr-x 3 jal jal    4096 mar 15 17:14 .
drwxrwxr-x 3 jal jal   20480 jan 20 21:08 ..
-rw-rw-r-- 1 jal jal     228 mar 15 17:14 190000.ctx
-rw-rw-r-- 1 jal jal   42545 mar 15 17:14 190000.graph.gexf
-rw-rw-r-- 1 jal jal   97335 mar 15 17:14 190000.headers_values.py
-rw-rw-r-- 1 jal jal    1088 jan 20 21:08 190000.heap.pointers.offsets
-rw-rw-r-- 1 jal jal    1088 jan 20 21:08 190000.heap.pointers.values
-rw-rw-r-- 1 jal jal    1120 jan 20 21:08 190000.mchunks.addrs
-rw-rw-r-- 1 jal jal    1120 jan 20 21:08 190000.mchunks.sizes
-rw-rw-r-- 1 jal jal    3337 mar 15 17:14 190000.strings
-rw-rw-r-- 1 jal jal     228 mar 15 17:14 1a0000.ctx
-rw-rw-r-- 1 jal jal     318 mar 15 17:14 1a0000.graph.gexf
-rw-rw-r-- 1 jal jal       0 mar 15 17:14 1a0000.headers_values.py
-rw-rw-r-- 1 jal jal      80 jan 20 21:08 1a0000.heap.pointers.offsets
-rw-rw-r-- 1 jal jal      80 jan 20 21:08 1a0000.heap.pointers.values
-rw-rw-r-- 1 jal jal       0 mar 15 17:14 1a0000.strings
-rw-rw-r-- 1 jal j

In [6]:
!cat ../test/dumps/vol/zeus.vmem.856.dump/cache/*.strings| grep -a http

0xc64e8,64,u'http://193.104.41.75/cbd/75.bro\x00'
0xc32d98,32,'http://193.104.41.75/cbd/75.bro '
0xc329f8,64,'Ahttps://onlineeast#.bankofamerica.com/cgi-bin/ias/*/GotoWelcome'


Mhh interesting string... I wonder what memory chunk was that allocated in.

In [10]:
!haystack-reverse-show ../test/dumps/vol/zeus.vmem.856.dump 0xc64e8


class struct_c64e8(ctypes.Structure):  # rlevel:50 SIG:T64 size:64
  _fields_ = [ 
	( 'utf16_0' , ctypes.c_char*64 ), #  # ctypes.c_char: u'http://193.104.41.75/cbd/75.bro\x00'
 ]




Ah, that make sense.. It's a classic utf16 string. The whole allocated memory chunk is being used for a string.

Lets look at the bytes behind the scene.

In [13]:
!haystack-reverse-hex ../test/dumps/vol/zeus.vmem.856.dump 0xc64e8

'h\x00t\x00t\x00p\x00:\x00/\x00/\x001\x009\x003\x00.\x001\x000\x004\x00.\x004\x001\x00.\x007\x005\x00/\x00c\x00b\x00d\x00/\x007\x005\x00.\x00b\x00r\x00o\x00\x00\x00'


I wonder if this record was referenced in some other record...
Maybe we can find a parent record that points to this string...

In [14]:
!haystack-reverse-parents ../test/dumps/vol/zeus.vmem.856.dump 0xc64e8

None


Tough luck... What about the others ?

In [15]:
!haystack-reverse-parents ../test/dumps/vol/zeus.vmem.856.dump 0xc32d98

None


In [16]:
!haystack-reverse-parents ../test/dumps/vol/zeus.vmem.856.dump 0xc329f8

#0xc31e90

class struct_c31e90(ctypes.Structure):  # rlevel:50 SIG:P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4P4a8 size:336
  _fields_ = [ 
	( 'ptr_string_c32000_0' , ctypes.c_void_p ), # @ 0x00c32000 # [vol_mapping_058]
	( 'ptr_string_c32020_4' , ctypes.c_void_p ), # @ 0x00c32020 # [vol_mapping_058]
	( 'ptr_string_c32040_8' , ctypes.c_void_p ), # @ 0x00c32040 # [vol_mapping_058]
	( 'ptr_string_c32060_12' , ctypes.c_void_p ), # @ 0x00c32060 # [vol_mapping_058]
	( 'ptr_string_c320a8_16' , ctypes.c_void_p ), # @ 0x00c320a8 # [vol_mapping_058]
	( 'ptr_string_c320d0_20' , ctypes.c_void_p ), # @ 0x00c320d0 # [vol_mapping_058]
	( 'ptr_string_c320f8_24' , ctypes.c_void_p ), # @ 0x00c320f8 # [vol_mapping_058]
	( 'ptr_string_c32120_28' , ctypes.c_void_p ), # @ 0x00c32120 # [vol_mapping_058]
	( 'ptr_string_c32148_32' , ctypes.c_void_p ), # @ 0x00c32148 # [vol_mapping_058]
	( 'pt

That looks interesting. A record made of 82x 4-bytes pointers and some trailings zeroes/padding.

Let's see if we can check that out with haystack CLI.


In [17]:
!cat ../test/structures/zeus/records.py

import ctypes


class array_of_pointers(ctypes.Structure):
    _fields_ = [('array', ctypes.CString*82)]


class array_of_wcharp(ctypes.Structure):
    _fields_ = [('array', ctypes.CWString*82)]


So, due to a little monkey patching, there is a CString ctypes types available in the haystack ctypes module.

In [18]:
!haystack-show ../test/dumps/vol/zeus.vmem.856.dump test.structures.zeus.records.array_of_pointers 0xc31e90

[# --------------- 0x1 
{ # <array_of_pointers at 0xc31e90>
"array": [
	"l" , # (CString)
	"u" , # (CString)
	"l" , # (CString)
	"s" , # (CString)
	"w" , # (CString)
	"s" , # (CString)
	"e" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"_" , # (CString)
	"s" , # (CString)
	"U" , # (CString)
	"s" , # (CString)
	"s" , # (CString)
	"u" , # (CString)
	"software" , # (CString)
	"system" , # (CString)
	"%" , # (CString)
	"%" , # (CString)
	"n" , # (CString)
	"o" , # (CString)
	"z" , # (CString)
	"*%u.%u.%u.%u*" , # (CString)
	"winsta0" , # (CString)
	"default" , # (CString)
	"gdiplus.dll" , # (CString)
	"ole32.dll" , # (CString)
	"gdi32.dll" , # (CString)
	"DISPLAY" , # (CString)
	"GdiplusStartup" , # (CString)
	"GdiplusShutdown" , # (CString)
	"GdipCreateBitmapFromHBITMAP" , # (CString)
	"GdipDisposeImage" , # (CString)
	"GdipGetImageEncodersSize" , # (CString)
	"GdipGetImageEncoders" , # (CString)
	"GdipSaveImageToStream" , # (CString)
	"CreateS

Oh, that is pretty good...
but it seems the first few strings are not quite right..

In [19]:
!haystack-reverse-hex ../test/dumps/vol/zeus.vmem.856.dump 0x00c32000
!haystack-reverse-show ../test/dumps/vol/zeus.vmem.856.dump 0x00c32000

'l\x00o\x00w\x00s\x00e\x00c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

class string_c32000(ctypes.Structure):  # rlevel:50 SIG:T12a12 size:24
  _fields_ = [ 
	( 'utf16_0' , ctypes.c_char*12 ), #  # ctypes.c_char: u'lowsec'
	( 'zerroes_12' , ctypes.c_ubyte*12 ), # # array
 ]




Mhh, it seems the first few strings are utf16 strings. Lets try with a Wide char string type.


In [20]:
!haystack-show ../test/dumps/vol/zeus.vmem.856.dump test.structures.zeus.records.array_of_wcharp 0xc31e90  

[# --------------- 0x1 
{ # <array_of_wcharp at 0xc31e90>
"array": [
	"l o w s e c" , # (CWString)
	"u s e r . d s" , # (CWString)
	"l o c a l . d s" , # (CWString)
	"s d r a 6 4 . e x e" , # (CWString)
	"w i n l o g o n . e x e" , # (CWString)
	"s v c h o s t . e x e" , # (CWString)
	"e x p l o r e r . e x e" , # (CWString)
	"_ A V I R A _ 2 1 1 0" , # (CWString)
	"_ A V I R A _ 2 1 0 1" , # (CWString)
	"_ A V I R A _ 2 1 0 8" , # (CWString)
	"_ A V I R A _ 2 1 0 9" , # (CWString)
	"_ A V I R A _ 2 1 0 9 9" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s   n t \ c u r r e n t v e r s i o n \ n e t w o r k" , # (CWString)
	"U I D" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s   n t \ c u r r e n t v e r s i o n \ w i n l o g o n" , # (CWString)
	"s o f t w a r e \ m i c r o s o f t \ w i n d o w s \ c u r r e n t v e r s i o n \ r u n" , # (CWString)
	"u s e r i n i t" , # (CWString)
	"software" , # (CWString)
	"system" , # (CWString)
	"% 0 8

Well, that more or less works.


Ultimately, you might want to clean the cache ( REMOVES ALL ANALYSIS FILES )